In [2]:
from jinja2 import Environment, FileSystemLoader
import pandas as pd
from pathlib import Path

def generate_report(company: str, year: int):
    base_path = Path(f"data/processed/{company}")
    env = Environment(loader=FileSystemLoader("templates"))
    template = env.get_template("report.html")

    profile = pd.read_csv(base_path / "profile.csv").iloc[0]
    people = pd.read_csv(base_path / "people.csv")
    financials = pd.read_csv(base_path / "financials.csv")
    financials = financials[financials["quarter"].str.contains(str(year))]
    market = pd.read_csv(base_path / "market_data.csv", parse_dates=["date"])
    market = market[market["date"].dt.year == year].sort_values("date")

    # Rebase and plot
    if not market.empty:
        market["rebased"] = 100 * market["last_price"] / market["last_price"].iloc[0]

        import matplotlib.pyplot as plt
        plt.figure(figsize=(8, 3))
        plt.plot(market["date"], market["rebased"])
        plt.title(f"{company} Stock Price - {year}")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(base_path / f"chart_{year}.png")
        plt.close()

    html = template.render(
        company=company,
        year=year,
        profile=profile,
        people=people.to_dict(orient="records"),
        financials=financials.to_dict(orient="records")
    )

    with open(base_path / f"report_{year}.html", "w") as f:
        f.write(html)


In [3]:
generate_report("Charles-Jackson", 2020)